In [ ]:
#!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt


--2025-09-24 10:00:56--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.2’

input.txt.2         100%[===================>]   1.06M  --.-KB/s    in 0.09s   

2025-09-24 10:00:57 (12.4 MB/s) - ‘input.txt.2’ saved [1115394/1115394]



In [2]:
with open('input.txt','r',encoding= 'utf-8') as f:
    text = f.read()
    

In [3]:
print('length of dataset in characters:', len(text))

length of dataset in characters: 1115394


In [4]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(len(chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
#create a mapping from characters to integers
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] #simple character level encoding
decode = lambda l: ''.join([itos[i]for i in l])

print(encode("Hi there"))
print(decode(encode("hi there")))

[20, 47, 1, 58, 46, 43, 56, 43]
hi there


In [7]:
import torch
data = torch.tensor(encode(text),dtype=torch.long)
print(data.shape)

torch.Size([1115394])


In [8]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]
print("train data len: ", len(train_data))
print("val data len ", len(val_data))


train data len:  1003854
val data len  111540


In [9]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")
    

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [11]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size,(batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets')
print(yb.shape)
print(yb)

print('-------')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target is : {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-------
when input is [24] the target is : 43
when input is [24, 43] the target is : 58
when input is [24, 43, 58] the target is : 5
when input is [24, 43, 58, 5] the target is : 57
when input is [24, 43, 58, 5, 57] the target is : 1
when input is [24, 43, 58, 5, 57, 1] the target is : 46
when input is [24, 43, 58, 5, 57, 1, 46] the target is : 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target is : 39
when input is [44] the target is : 53
when input is [44, 53] the target is : 56
when input is [44, 53, 56] the target is : 1
when input is [44, 53, 56, 1] the target is : 58
when input is [44, 5

In [12]:
#implementing the bigram model
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        #each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)
    
    def forward(self,idx,targets=None):
        
        #idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
        
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            
            loss = F.cross_entropy(logits,targets)
        
        return logits, loss
    
    def generate(self,idx,max_new_tokens):
        #idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            #get the predictions
            logits, loss = self(idx)
            #focus on the last time step
            logits = logits[:,-1,:]#becomes(B,C)
            #apply softmax to get the probabilites
            probs = F.softmax(logits,dim=-1) #(B,C)
            #sample from the distribution
            idx_next = torch.multinomial(probs,num_samples=1)# (B,1)
            #append sampled index to the running sequence
            idx = torch.cat((idx,idx_next),dim=1) #(B,T+1)
            
        return idx

BigramModel = BigramLanguageModel(vocab_size)
logits,loss = BigramModel(xb,yb)
print(logits.shape)
print(loss)


idx = torch.zeros((1,1),dtype=torch.long)
print(decode(BigramModel.generate(idx,max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [13]:
optimizer = torch.optim.AdamW(BigramModel.parameters(),lr=1e-3)

In [26]:
batch_size = 32

for step in range(1000):
    xb,yb = get_batch('train')
    logits,loss = BigramModel(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss)

tensor(2.3825, grad_fn=<NllLossBackward0>)


In [27]:
print(decode(BigramModel.generate(idx,max_new_tokens=100)[0].tolist()))




Fore, anenoweld bt; poup ecored be af FomeatscakevexENRI'des t nd cadeld cer:
Weale? ighalveangry 
